Upload the raw dataset and/or any intermediate artifacts.

In [ ]:
import google.colab
import os

UPLOAD_ROOT = '/content'
DATA_FILE = 'S1_File.txt'

if not os.path.isfile(os.path.join(UPLOAD_ROOT, DATA_FILE)):
  google.colab.files.upload()

assert os.path.isfile(os.path.join(UPLOAD_ROOT, DATA_FILE))

Parse the raw dataset.

In [ ]:
import pandas as pd

COLUMN_PATIENT = 'PID'
COLUMN_DATE = 'DAY_ID'
COLUMN_DIAGNOSIS = 'DX_GROUP_DESCRIPTION'
COLUMN_SERVICE = 'SERVICE_LOCATION'

RawData = pd.read_csv(os.path.join(UPLOAD_ROOT, DATA_FILE), sep='\t')
assert COLUMN_PATIENT in RawData.columns
assert COLUMN_DATE in RawData.columns
assert COLUMN_DIAGNOSIS in RawData.columns
assert COLUMN_SERVICE in RawData.columns
RawData.head()

,PID,DAY_ID,DX_GROUP_DESCRIPTION,SERVICE_LOCATION,OP_DATE
0,1,73888,ANGINA PECTORIS,DOCTORS OFFICE,74084
1,1,73888,MONONEURITIS OF UPPER LIMB AND MONONEURITIS MU...,DOCTORS OFFICE,74084
2,1,73888,SYMPTOMS INVOLVING RESPIRATORY SYSTEM AND OTHE...,DOCTORS OFFICE,74084
3,1,73880,ACUTE APPENDICITIS,INPATIENT HOSPITAL,74084
4,1,73880,DIABETES MELLITUS,INPATIENT HOSPITAL,74084


Tally the occurrence of different diagnosis in the raw dataset.

In [ ]:
DiagnosisTally = RawData.groupby(COLUMN_DIAGNOSIS).size().to_frame('COUNT').sort_values(by='COUNT', ascending=False).reset_index()
DiagnosisTally.head()

,DX_GROUP_DESCRIPTION,COUNT
0,CARDIAC DYSRHYTHMIAS,35522
1,ESSENTIAL HYPERTENSION,32998
2,HEART FAILURE,32117
3,DIABETES MELLITUS,30376
4,DISORDERS OF LIPOID METABOLISM,20157


Diagnosis description will be used as vocab list. The number of distinct diagnosis is huge.

In [ ]:
DiagnosisTally.shape[0]

1412

Remove diagnosis words with rare occurrence to optimize training and reduce noise. The vocab list is now much smaller.

In [ ]:
RARE_OCCURRENCE = 100

DiagnosisTally = DiagnosisTally[DiagnosisTally['COUNT'] > RARE_OCCURRENCE]
DiagnosisTally = DiagnosisTally.sort_values(by='COUNT', ascending=False)
DiagnosisTally.shape[0]

490

Constructing a word to index dictionary `Vocab`.

In [ ]:
ARTIFACT_VOCAB = 'Artifact_Vocab.csv'

if not os.path.isfile(os.path.join(UPLOAD_ROOT, ARTIFACT_VOCAB)):
  Vocab = DiagnosisTally.drop(columns=['COUNT'])
  Vocab = Vocab.reset_index()
  Vocab['index'] = Vocab['index']
  Vocab.to_csv(ARTIFACT_VOCAB, index=False)

Vocab = pd.read_csv(os.path.join(UPLOAD_ROOT, ARTIFACT_VOCAB))
assert 'index' in Vocab
assert COLUMN_DIAGNOSIS in Vocab
Vocab.head()

,index,DX_GROUP_DESCRIPTION
0,0,CARDIAC DYSRHYTHMIAS
1,1,ESSENTIAL HYPERTENSION
2,2,HEART FAILURE
3,3,DIABETES MELLITUS
4,4,DISORDERS OF LIPOID METABOLISM


Encode diagnosis data using `Vocab`.

In [ ]:
import ast

ARTIFACT_ENCODED_RAW_DATA = 'Artifact_EncodedRawData.csv'

if not os.path.isfile(os.path.join(UPLOAD_ROOT, ARTIFACT_ENCODED_RAW_DATA)):
  EncodedRawData = RawData.join(other=Vocab.set_index(COLUMN_DIAGNOSIS), on=COLUMN_DIAGNOSIS, how='inner')
  EncodedRawData = EncodedRawData.groupby([COLUMN_PATIENT, COLUMN_DATE])['index'].apply(lambda x: list(set(list(x)))).reset_index()
  EncodedRawData = EncodedRawData.rename(columns={'index': COLUMN_DIAGNOSIS})
  EncodedRawData.to_csv(ARTIFACT_ENCODED_RAW_DATA, index=False)

EncodedRawData = pd.read_csv(os.path.join(UPLOAD_ROOT, ARTIFACT_ENCODED_RAW_DATA), converters={COLUMN_DIAGNOSIS: ast.literal_eval})
assert COLUMN_PATIENT in EncodedRawData
assert COLUMN_DATE in EncodedRawData
assert COLUMN_DIAGNOSIS in EncodedRawData
EncodedRawData.head()

,PID,DAY_ID,DX_GROUP_DESCRIPTION
0,1,73546,[26]
1,1,73564,"[12, 5]"
2,1,73571,"[16, 51, 12, 85]"
3,1,73586,"[3, 4, 7, 18, 479]"
4,1,73617,[18]


Extract hospitalization events from raw data.

In [ ]:
SERVICE_HOSPITALIZATION = 'INPATIENT HOSPITAL'

HospitalizationEvents = RawData[RawData[COLUMN_SERVICE] == SERVICE_HOSPITALIZATION]
HospitalizationEvents = HospitalizationEvents.drop(columns=[COLUMN_SERVICE])
HospitalizationEvents = HospitalizationEvents.groupby([COLUMN_PATIENT, COLUMN_DATE]).size().to_frame('DIAGNOSIS_COUNT').reset_index()
HospitalizationEvents = HospitalizationEvents.drop(columns='DIAGNOSIS_COUNT')
HospitalizationEvents.head()

,PID,DAY_ID
0,1,73874
1,1,73879
2,1,73880
3,1,74043
4,2,74195


For every patients' event, identify if there exists a readmission within 30 days. Build a readmission ground truth dataset.

In [ ]:
ARTIFACT_DATASET = 'Artifact_Dataset.csv'
READMISSION_DAYS = 30

if not os.path.isfile(os.path.join(UPLOAD_ROOT, ARTIFACT_DATASET)):
  def hasReadmission(hospitalizationEvents, patient, date):
    patientEvents = hospitalizationEvents[hospitalizationEvents[COLUMN_PATIENT] == patient]
    return patientEvents[(date < patientEvents[COLUMN_DATE]) & (patientEvents[COLUMN_DATE] < (date + READMISSION_DAYS))].shape[0] > 0
  Dataset = EncodedRawData
  Dataset['READMISSION'] = Dataset.apply(lambda x: int(hasReadmission(HospitalizationEvents, x[COLUMN_PATIENT], x[COLUMN_DATE])), axis=1)
  Dataset.to_csv(ARTIFACT_DATASET, index=False)

Dataset = pd.read_csv(os.path.join(UPLOAD_ROOT, ARTIFACT_DATASET), converters={COLUMN_DIAGNOSIS: ast.literal_eval})
assert COLUMN_PATIENT in Dataset
assert COLUMN_DATE in Dataset
assert COLUMN_DIAGNOSIS in Dataset
assert 'READMISSION' in Dataset
Dataset.head()

,PID,DAY_ID,DX_GROUP_DESCRIPTION,READMISSION
0,1,73546,[26],0
1,1,73564,"[12, 5]",0
2,1,73571,"[16, 51, 12, 85]",0
3,1,73586,"[3, 4, 7, 18, 479]",0
4,1,73617,[18],0


Build the dataset `Documents` and the ground truth `Labels`.

`Documents` is a list, each entry is the information of 1 single `Patient`.

```
Documents = [Patient_1, Patient_2]
```

Each `Patient_i` is a list, each entry is the information of 1 single `Visit`.

```
Patient_i = [Visit_1, Visit_2]
```

Each `Visit_i` is a list of encoded diagnosis.

```
Visit_i = [Diag_1, Diag_2]
```

In [ ]:
Documents = Dataset.groupby([COLUMN_PATIENT])[COLUMN_DIAGNOSIS].apply(list).reset_index()[COLUMN_DIAGNOSIS].tolist()
Labels = Dataset.groupby([COLUMN_PATIENT])['READMISSION'].max().reset_index()['READMISSION'].tolist()
len(Documents)

3000

Split training and testing sets.

In [ ]:
import sklearn.model_selection
import torch.utils.data

RANDOM_STATE = 42
TEST_SIZE = 0.2

class PatientDataset(torch.utils.data.Dataset):
  def __init__(self, documents, labels):
    self.x = documents
    self.y = labels
  def __len__(self):
    return len(self.x)
  def __getitem__(self, index):
    return self.x[index], self.y[index]

x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(Documents, Labels, test_size=TEST_SIZE, random_state=RANDOM_STATE)
TrainingDataset = PatientDataset(x_train, y_train)
TestingDataset = PatientDataset(x_test, y_test)
len(TrainingDataset)

2400

Define the dataset loader.

In [ ]:
import torch.utils.data
import torch

BATCH_SIZE = 1

def collateFunc(data):
  sequences, labels = zip(*data)
  numPatients = len(sequences)
  numVisits = [len(patient) for patient in sequences]
  maxNumVisits = max(numVisits)
  x = torch.zeros((numPatients, maxNumVisits, DiagnosisTally.shape[0]), dtype=torch.float)
  for patientIndex, patient in enumerate(sequences):
    for visitIndex, visit in enumerate(patient):
      for diagnosis in visit:
        x[patientIndex][visitIndex][diagnosis] = 1
  x_masks = torch.sum(x, dim=-1) > 0
  y = torch.tensor(labels, dtype=torch.float)
  return x, x_masks, y

TrainingLoader = torch.utils.data.DataLoader(TrainingDataset, shuffle=True, batch_size=BATCH_SIZE, collate_fn=collateFunc)
TestingLoader = torch.utils.data.DataLoader(TestingDataset, batch_size=BATCH_SIZE, collate_fn=collateFunc)
len(TrainingLoader)

2400

Define the CONTENT model.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

HIDDEN_SIZE = 200
EMBEDDING_DIM = 100

class GruNet(nn.Module):
  def __init__(self, inputSize):
    super().__init__()
    self.gru = nn.GRU(input_size=inputSize, hidden_size=HIDDEN_SIZE, batch_first=True)
    self.fc = nn.Linear(in_features=HIDDEN_SIZE, out_features=1)
  def forward(self, x):
    _, h = self.gru(x)
    out = self.fc(h)
    return out

class RecogNet(nn.Module):
  def __init__(self, inputSize):
    super().__init__()
    self.fc1 = nn.Linear(in_features=inputSize, out_features=HIDDEN_SIZE)
    self.fc2 = nn.Linear(in_features=HIDDEN_SIZE, out_features=HIDDEN_SIZE)
    self.fc3 = nn.Linear(in_features=HIDDEN_SIZE, out_features=1)
  def forward(self, x):
    out = F.relu(self.fc1(x))
    out = F.relu(self.fc2(out))
    out = F.relu(self.fc3(out))
    return out

class ContentNet(nn.Module):
  def __init__(self):
    super().__init__()
    self.rnn = GruNet(Vocab.shape[0])
    self.recog = RecogNet(Vocab.shape[0])
    self.fc = nn.Linear(in_features=2, out_features=1)
  def forward(self, x, masks):
    batch_size = x.shape[0]
    rnnOut = self.rnn(x)
    recogOut = self.rnn(x)
    logits = self.fc(torch.cat((rnnOut, recogOut), dim=-1))
    probs = torch.sigmoid(logits)
    return probs.view(batch_size)

model = ContentNet()
model

ContentNet(
  (rnn): GruNet(
    (gru): GRU(490, 200, batch_first=True)
    (fc): Linear(in_features=200, out_features=1, bias=True)
  )
  (recog): RecogNet(
    (fc1): Linear(in_features=490, out_features=200, bias=True)
    (fc2): Linear(in_features=200, out_features=1, bias=True)
  )
  (fc): Linear(in_features=2, out_features=1, bias=True)
)

Model training.

In [ ]:
EPOCHS = 1

def eval(model, dataLoader):
  model.eval()
  y_pred = torch.LongTensor()
  y_score = torch.Tensor()
  y_true = torch.LongTensor()
  for x, masks, y in dataLoader:
    y_hat = model(x, masks)
    y_score = torch.cat((y_score, y_hat.detach().to('cpu')), dim=0)
    y_hat = (y_hat > 0.5).int()
    y_pred = torch.cat((y_pred, y_hat.detach().to('cpu')), dim=0)
    y_true = torch.cat((y_true, y.detach().to('cpu')), dim=0)
  p, r, f, _ = sklearn.metrics.precision_recall_fscore_support(y_true=y_true, y_pred=y_pred, average='binary')
  roc_auc = sklearn.metrics.roc_auc_score(y_true=y_true, y_score=y_score)
  return p, r, f, roc_auc

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), 0.001)

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0
    for x, x_masks, y in TrainingLoader:
        loss = None
        optimizer.zero_grad()
        outputs = model(x, x_masks)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss = train_loss / len(TrainingLoader)
    print('Epoch: {} \t Training Loss: {:.6f}'.format(epoch+1, train_loss))
    p, r, f, roc_auc = eval(model, TrainingLoader)
    print('Epoch: {} \t Validation p: {:.2f}, r:{:.2f}, f: {:.2f}, roc_auc: {:.2f}'.format(epoch+1, p, r, f, roc_auc))

Epoch: 1 	 Training Loss: 0.340308
Epoch: 1 	 Validation p: 0.90, r:1.00, f: 0.95, roc_auc: 0.64
